In [1]:
from keras import models, layers, losses, optimizers, metrics
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [3]:
train_dir = '/media/shuvendu/Projects/Datasets/bangla/train'
test_dir = '/media/shuvendu/Projects/Datasets/bangla/test'

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(64, 64), 
    batch_size=32
)

test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(64, 64), 
    batch_size=32
)

Found 12000 images belonging to 50 classes.
Found 3000 images belonging to 50 classes.


In [4]:
for data_batch, label_batch in train_generator:
    print(data_batch.shape)
    print(label_batch.shape)
    break

(32, 64, 64, 3)
(32, 50)


# Deep neural network

In [19]:
model = models.Sequential()

model.add(layers.Flatten(input_shape=(64, 64, 3)))
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(50, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_8 (Flatten)          (None, 12288)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 1024)              12583936  
_________________________________________________________________
dense_10 (Dense)             (None, 50)                51250     
Total params: 12,635,186
Trainable params: 12,635,186
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
model.compile(
    optimizer=optimizers.RMSprop(lr=1e-4), 
    loss=losses.categorical_crossentropy, 
    metrics=[metrics.categorical_accuracy]
)

history = model.fit_generator(train_generator, 
                              steps_per_epoch=375, 
                              epochs=30, 
                              validation_data=test_generator, 
                              validation_steps=95)

Epoch 1/30
375/375 [==============================] - 6s - loss: 3.0744 - categorical_accuracy: 0.2232 - val_loss: 2.5420 - val_categorical_accuracy: 0.3259
Epoch 2/30
375/375 [==============================] - 5s - loss: 2.0645 - categorical_accuracy: 0.4482 - val_loss: 1.9759 - val_categorical_accuracy: 0.4898
Epoch 3/30
375/375 [==============================] - 5s - loss: 1.7460 - categorical_accuracy: 0.5297 - val_loss: 1.9024 - val_categorical_accuracy: 0.4908
Epoch 4/30
375/375 [==============================] - 5s - loss: 1.5648 - categorical_accuracy: 0.5757 - val_loss: 1.8918 - val_categorical_accuracy: 0.5059
Epoch 5/30
375/375 [==============================] - 5s - loss: 1.4262 - categorical_accuracy: 0.6111 - val_loss: 1.7281 - val_categorical_accuracy: 0.5472
Epoch 6/30
  5/375 [..............................] - ETA: 5s - loss: 1.2717 - categorical_accuracy: 0.6750

# Convolutional neural network

In [11]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(layers.MaxPool2D(2, 2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPool2D(2, 2))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPool2D(2, 2))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(50, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 12, 12, 128)       73856     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 6, 6, 128)         0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 4608)              0         
__________

In [12]:
# Compile the model
model.compile(
    optimizer=optimizers.RMSprop(lr=1e-4), 
    loss=losses.categorical_crossentropy, 
    metrics=[metrics.categorical_accuracy]
)

In [13]:
history = model.fit_generator(train_generator, 
                              steps_per_epoch=375, 
                              epochs=30, 
                              validation_data=test_generator, 
                              validation_steps=95)

Epoch 1/30
375/375 [==============================] - 22s - loss: 2.8325 - categorical_accuracy: 0.2823 - val_loss: 1.8247 - val_categorical_accuracy: 0.5145
Epoch 2/30
375/375 [==============================] - 5s - loss: 1.3321 - categorical_accuracy: 0.6377 - val_loss: 1.0710 - val_categorical_accuracy: 0.7042
Epoch 3/30
375/375 [==============================] - 5s - loss: 0.7897 - categorical_accuracy: 0.7822 - val_loss: 0.7529 - val_categorical_accuracy: 0.7919
Epoch 4/30
375/375 [==============================] - 4s - loss: 0.5368 - categorical_accuracy: 0.8447 - val_loss: 0.6490 - val_categorical_accuracy: 0.8183
Epoch 5/30
375/375 [==============================] - 5s - loss: 0.3837 - categorical_accuracy: 0.8898 - val_loss: 0.5530 - val_categorical_accuracy: 0.8506
Epoch 6/30
375/375 [==============================] - 4s - loss: 0.2743 - categorical_accuracy: 0.9223 - val_loss: 0.5283 - val_categorical_accuracy: 0.8668
Epoch 7/30
375/375 [==============================] - 5s 

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training losses')
plt.plot(epochs, val_loss, 'b', label='Validation losses')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# Applying image augmentation

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=15, 
                                   width_shift_range=0.05, 
                                   height_shift_range=0.05, 
                                   shear_range=0.05,
                                   zoom_range=0.1, 
                                   horizontal_flip=False)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(150, 150), 
    batch_size=32
)

test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(150, 150), 
    batch_size=32
)

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPool2D(2, 2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPool2D(2, 2))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPool2D(2, 2))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPool2D(2, 2))

model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(84, activation='softmax'))

model.summary()

In [ ]:
model.compile(
    optimizer=optimizers.RMSprop(lr=1e-4), 
    loss=losses.categorical_crossentropy, 
    metrics=[metrics.categorical_accuracy]
)

In [ ]:
history = model.fit_generator(train_generator, 
                              steps_per_epoch=4200, 
                              epochs=30, 
                              validation_data=test_generator, 
                              validation_steps=900)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training losses')
plt.plot(epochs, val_loss, 'b', label='Validation losses')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model.save('image_augmentation_model.h5')

### Observtion

Image augmentation gives 1% increase in valdiation accuray giving 89% accuracy. but it has some ridiculous validation_acc > train_acc
So, image augmentation is not a good choice. Let's try using pretrained model with no image augmentation. if that doesn't work well then deep cnn with drop out having around 88% accuracy was the best option so far

# Try using pretrained model (VGG16 on imagenet)
using imagenet dataset is not an ideal case. But give it a shot and see how the low level feature deection help

In [ ]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                 include_top=False,
                 input_shape=(150, 150, 3))

In [ ]:
conv_base.summary()

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(150, 150), 
    batch_size=32
)

test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(150, 150), 
    batch_size=32
)

In [ ]:
model = models.Sequential()

model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(84, activation='softmax'))
model.summary()

In [ ]:
conv_base.trainable = False
print('This is the number of trainable weights before freezing the conv base:', len(model.trainable_weights))

In [ ]:
model.compile(
    optimizer=optimizers.RMSprop(lr=1e-4), 
    loss=losses.categorical_crossentropy, 
    metrics=[metrics.categorical_accuracy]
)

In [ ]:
history = model.fit_generator(train_generator, 
                              steps_per_epoch=4200, 
                              epochs=10, 
                              validation_data=test_generator, 
                              validation_steps=900)

In [ ]:
model.save('dense_layer_on_pretrained_vgg16.h5')

# Fine-tuning
As the pretrained model improves the accuracy. let's try fine tuning it for the last blocks

In [ ]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
model.compile(
    optimizer=optimizers.RMSprop(lr=1e-4), 
    loss=losses.categorical_crossentropy, 
    metrics=[metrics.categorical_accuracy]
)

history = model.fit_generator(train_generator, 
                              steps_per_epoch=4200, 
                              epochs=10, 
                              validation_data=test_generator, 
                              validation_steps=900)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training losses')
plt.plot(epochs, val_loss, 'b', label='Validation losses')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model.save('finetuning_on_pretrained_vgg16.h5')

# Conclusion

With the pretrained vgg26 model validation accuracy upto 92% is achieved